# API
Выполнила: Остапчук Таисия Сергеевна, 121-ПМо

In [ ]:
import pandas as pd
import requests
import json

# Собираем вакансии
Проходимся по страницам и по заданным параметрам собираем необходимую информацию



In [ ]:
def get_vacancies():
    url = 'https://api.hh.ru/vacancies'
    jsons_list = []

    for num_page in range(20):
        params = {'text': 'разработчик', 'per_page': '100', 'page': num_page}
        src = requests.get(url, params=params)
        jsons_list.append(src.json())

    return jsons_list

In [ ]:
vacancies = get_vacancies()

# Создаем DataFrame
DataFrame с заголовками, которые соответствуют информации из API

In [ ]:
data = pd.DataFrame(columns=['name', 'alternate_url',  'created_at', 'archived', 'experience',
                            'salary_from', 'salary_to', 'salary_currency', 'salary_how', 'gross',
                            'work_format', 'working_hours', 'work_days', 'employment_form',
                            'accept_temporary', 'has_test', 'internship', 'night_shifts',
                            'employer', 'employer_url', 'accredited_it',
                            'city', 'street', 'building'])
data

,name,alternate_url,created_at,archived,experience,salary_from,salary_to,salary_currency,salary_how,gross,...,accept_temporary,has_test,internship,night_shifts,employer,employer_url,accredited_it,city,street,building


# Собираем информацию в DataFrame
По полученным данным из API заполняем DataFrame

In [ ]:
for page in vacancies:
  for vacancy in page['items']:
      id = vacancy['id']
      name = vacancy['name']

      if vacancy['salary_range']:
          salary_from = vacancy['salary_range'].get('from', None)
          salary_to = vacancy['salary_range'].get('to', None)
          salary_currency = vacancy['salary_range'].get('currency', None)
          gross = vacancy['salary_range'].get('gross', False)
          salary_how = vacancy['salary_range']['mode']['name'] if vacancy['salary_range']['mode'] else None
      else:
          salary_from, salary_to, salary_currency, gross, salary_how = None, None, None, None, None

      accept_temporary = vacancy.get('accept_temporary', None)

      if vacancy['address']:
          city = vacancy['address'].get('city', None)
          street = vacancy['address'].get('street', None)
          building = vacancy['address'].get('building', None)
      else:
          city, street, building = None, None, None

      alternate_url = vacancy.get('alternate_url', None)
      archived = vacancy.get('archived', None)
      created_at = pd.to_datetime(vacancy.get('created_at', None))

      if vacancy['employer']:
          employer = vacancy['employer'].get('name', None)
          employer_url = vacancy['employer'].get('alternate_url', None)
          accredited_it = vacancy['employer'].get('accredited_it_employer', None)
      else:
          employer, employer_url, accredited_it = None, None, None

      experience = vacancy['experience']['name'] if vacancy['experience'] else None
      has_test = vacancy.get('has_test', None)
      internship = vacancy.get('internship', None)
      night_shifts = vacancy.get('night_shifts', None)

      employment_form = vacancy['employment_form']['name'] if vacancy['employment_form'] else None
      work_format = vacancy['work_format'][0]['name'] if vacancy['work_format'] else None
      work_days = vacancy['work_schedule_by_days'][0]['name'] if vacancy['work_schedule_by_days'] else None
      working_hours = vacancy['working_hours'][0]['name'] if vacancy['working_hours'] else None

      data.loc[id] = [
                      name, alternate_url,  created_at, archived, experience,
                      salary_from, salary_to, salary_currency, salary_how, gross,
                      work_format, working_hours, work_days, employment_form,
                      accept_temporary, has_test, internship, night_shifts,
                      employer, employer_url, accredited_it,
                      city, street, building]

# Посмотрим на данные



In [ ]:
data.sample(10)

,name,alternate_url,created_at,archived,experience,salary_from,salary_to,salary_currency,salary_how,gross,...,accept_temporary,has_test,internship,night_shifts,employer,employer_url,accredited_it,city,street,building
128599686,Инженер,https://hh.ru/vacancy/128599686,2025-12-10 06:26:38+03:00,False,От 1 года до 3 лет,88000,150000,RUR,За месяц,False,...,False,False,False,False,Научно-производственная фирма Микран,https://hh.ru/employer/1023318,False,Томск,проспект Кирова,51Д
128545168,Web-разработчик,https://hh.ru/vacancy/128545168,2025-12-08 17:21:05+03:00,False,От 1 года до 3 лет,150000,180000,RUR,За месяц,False,...,False,False,False,False,World Vision,https://hh.ru/employer/5069114,False,Москва,Чапаевский переулок,3
128163966,Диджитал-маркетолог (Digital-маркетолог),https://hh.ru/vacancy/128163966,2025-12-13 12:08:43+03:00,False,От 3 до 6 лет,150000,None,RUR,За месяц,False,...,False,False,False,False,ГОРОДСКАЯ СЛУЖБА РИТУАЛ,https://hh.ru/employer/1100388,False,Химки,улица Бабакина,5А
128714779,Middle Backend developer (C#/.net),https://hh.ru/vacancy/128714779,2025-12-12 18:13:56+03:00,False,От 3 до 6 лет,None,None,None,None,None,...,False,False,False,False,Nexign (АО Нэксайн),https://hh.ru/employer/6004,False,Санкт-Петербург,Уральская улица,4
128738342,Продакт-маркетолог в международный стартап,https://hh.ru/vacancy/128738342,2025-12-15 10:27:41+03:00,False,От 1 года до 3 лет,None,None,None,None,None,...,False,False,False,False,ЛИЦА,https://hh.ru/employer/3926495,False,None,None,None
128758989,Руководитель отдела разработки ПО,https://hh.ru/vacancy/128758989,2025-12-15 16:36:29+03:00,False,Более 6 лет,None,None,None,None,None,...,False,False,False,False,ТехноГрупп,https://hh.ru/employer/5069020,False,Москва,улица Щипок,9/26с1
128709717,Java Backend Developer (Вакансия только для гр...,https://hh.ru/vacancy/128709717,2025-12-12 15:40:04+03:00,False,От 3 до 6 лет,20000000,None,UZS,За месяц,True,...,False,False,False,False,ГУ ADLIYA ORGANLARI VA MUASSASALARIDA AXBOROT-...,https://hh.ru/employer/12424964,False,Ташкент,улица Тараса Шевченко,7
128741195,IT Sourcer (Researcher),https://hh.ru/vacancy/128741195,2025-12-15 11:13:34+03:00,False,Нет опыта,None,None,None,None,None,...,False,False,False,False,АксамИТ Мобайл Солюшнс,https://hh.ru/employer/3983252,False,Минск,проспект Победителей,7А
128685625,Sales manager,https://hh.ru/vacancy/128685625,2025-12-14 19:54:23+03:00,False,От 3 до 6 лет,1500,4000,EUR,За месяц,True,...,False,False,False,False,ONESOIL HOLDING AG,https://hh.ru/employer/12335104,False,None,None,None
128678982,Portfolio Development Lead,https://hh.ru/vacancy/128678982,2025-12-11 16:21:36+03:00,False,Более 6 лет,None,None,None,None,None,...,False,False,False,False,Pfizer,https://hh.ru/employer/4793,False,Москва,Пресненская набережная,10блокС
